<a href="https://colab.research.google.com/github/hamletbatista/sej/blob/master/Hands_on_Introduction_to_Machine_Learning_for_SEOs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Hands-on Introduction to Machine Learning for SEOs

Here is technical plan to generate the training dataset:

1. Extract: we will walk through my code to connect to Google Search Console and pull CTR data
2. Transform: then, we will fetch the pages titles and meta descriptions and normalize/clean them
3. Load: finally, we will populate a Google Sheet with the pages, titles, meta descriptions and CTRs

In [0]:
spreadsheetName="A Hands-on Introduction to Machine Learning for SEOs" #@param {type:"string"}
sitename="https://www.domain.com/" #@param {type:"string"}
client_id="client_id.json" #@param {type:"string"}


## Extracting from Google Search Console

First, there is some setup to download a client_id.json file our Python code can use to connect securely to Google Search Console.

1. Activate Search Console API in Compute Engine  https://console.cloud.google.com/apis/api/webmasters.googleapis.com/overview?project=&folder=&organizationId=
2. Create New Credentials / Help me choose (Search Console API, Other UI, User data)  https://console.cloud.google.com/apis/credentials/wizard?api=iamcredentials.googleapis.com&project=
3. Download client_id.json

In [0]:
#Next, we need to upload the file
from google.colab import files

files.upload()

In [0]:
# Go to Runtime > run after
!pip install git+https://github.com/joshcarty/google-searchconsole

In [0]:
# run once
import searchconsole
account = searchconsole.authenticate(client_config="client_id.json", serialize='credentials.json', flow="console")


Let's get our Search Console data

In [0]:
webproperty = account[sitename]

In [0]:
#let's build a pandas dataframe with the search console data
import pandas as pd

def get_search_console_data(webproperty, days=-7):
  if webproperty is not None:
    query = webproperty.query.range(start='today', days=days).dimension('page', 'query', 'country', 'device')

    r = query.get()
    df = pd.DataFrame(r.rows)
    return df

  print("Web property doesn't exist, please select a valid one from this list")
  print(account.webproperties)

  return None


In [0]:
df = get_search_console_data(webproperty)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104290 entries, 0 to 104289
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   page         104290 non-null  object 
 1   query        104290 non-null  object 
 2   country      104290 non-null  object 
 3   device       104290 non-null  object 
 4   clicks       104290 non-null  float64
 5   impressions  104290 non-null  float64
 6   ctr          104290 non-null  float64
 7   position     104290 non-null  float64
dtypes: float64(4), object(4)
memory usage: 6.4+ MB


In [0]:
df.head()

In [11]:
pages = list(set(df.page))
len(pages)

2707

## Extracting title and meta descriptions

Let's fetch the titles and meta descriptions

In [0]:
#let's install the library
!pip install requests-html

In [0]:
from requests_html import HTMLSession

def get_title_meta_description(page):
  session = HTMLSession()

  try:
    r = session.get(page)

    if r.status_code == 200:
      title = r.html.xpath('//title/text()')
      meta_description = r.html.xpath("//meta[@name='description']/@content")

      #Inner utility function
      def get_first(result):
        if len(result) == 0:
          return None
        else:
          return result[0]

      return {"title": get_first(title), "meta_description": get_first(meta_description)}

    else:
      print(f"Failed to fetch page: {page} with status code {r.status_code}")

  except:
    print(f"Failed to fetch page: {page}")
    return None


In [0]:
get_title_meta_description(pages[0])

In [0]:
# let's get all of them
titles_and_meta_descriptions=dict()



In [0]:
import time

for page in pages[:10]: # test on smaller number first
  print(f"Fetching page: {page}")
  titles_and_meta_descriptions[page] = get_title_meta_description(page)
  #add delay between requests
  time.sleep(1)

Let's see if the output is coming out Ok

In [0]:
titles_and_meta_descriptions

Now, let's get the full list and turn off printing to do it faster

In [0]:
import time

for page in pages: # running full list
  print(f"Fetching page: {page}")
  titles_and_meta_descriptions[page] = get_title_meta_description(page)
  #add delay between requests
  time.sleep(1)

## Creating a new dataframe



In [0]:
data = list()

for k, v in titles_and_meta_descriptions.items():
  #print (k,v)
  row = dict()
  if v is None:
    print(f"skipping page: {k}")
    continue 

  row["page"] = k
  row["title"] = v["title"]
  row["meta_description"] = v["meta_description"]

  data.append(row)


In [0]:
new_df = pd.DataFrame(data)

In [0]:
new_df.head()

In [0]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2591 entries, 0 to 2590
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   page              2591 non-null   object
 1   title             2039 non-null   object
 2   meta_description  1995 non-null   object
dtypes: object(3)
memory usage: 60.9+ KB


## Merge dataframes



In [0]:
merged_df=pd.merge(df, new_df, how="left", on="page")

In [0]:
merged_df.head()

In [0]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99495 entries, 0 to 99494
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   page              99495 non-null  object 
 1   query             99495 non-null  object 
 2   country           99495 non-null  object 
 3   device            99495 non-null  object 
 4   clicks            99495 non-null  float64
 5   impressions       99495 non-null  float64
 6   ctr               99495 non-null  float64
 7   position          99495 non-null  float64
 8   title             96639 non-null  object 
 9   meta_description  92949 non-null  object 
dtypes: float64(4), object(6)
memory usage: 8.3+ MB


In [0]:
merged_df.dropna().head()

## Keywords in title

https://stackoverflow.com/questions/31806695/when-to-use-which-fuzz-function-to-compare-2-strings
https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
https://github.com/seatgeek/fuzzywuzzy

In [0]:
!pip install fuzzywuzzy[speedup]

In [0]:
from fuzzywuzzy import fuzz

In [0]:
#remove empty rows
df = merged_df.dropna()

In [0]:
#df.apply(lambda row: fuzz.partial_ratio(row["query"], row["title"]), axis = 1)
df["query_in_title"] = df.apply(lambda row: fuzz.partial_ratio(row["query"], row["title"]), axis=1)


In [0]:
df.head()

## Populating our Google Sheet

In [0]:
#https://pypi.org/project/gspread-pandas/


In [0]:
!pip install git+https://github.com/hamletbatista/gspread-pandas.git@patch-1
#https://github.com/aiguofer/gspread-pandas/pull/47

In [0]:
# Authenticate (copy & paste key as detailed)
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials

In [0]:
from gspread_pandas import Spread, Client

In [0]:
creds = GoogleCredentials.get_application_default()

In [0]:
spread = Spread(spreadsheetName, creds=creds)


In [0]:
#https://gspread-pandas.readthedocs.io/en/latest/getting_started.html

In [32]:
# Display available worksheets
spread.sheets

[<Worksheet 'Sheet1' id:0>, <Worksheet 'Client' id:1763709549>]

In [0]:
# Save DataFrame to worksheet 'Client', create it first if it doesn't exist
spread.df_to_sheet(df, index=False, sheet='Client', start='A1', replace=True)